In [1]:
import pyNeuroChem as pyc
import hdnntools as hdt
import numpy as np
import time

from ase_interface import ANIENS
from ase_interface import ensemblemolecule

from ase.atoms import Atoms
from ase.optimize import BFGS, LBFGS

import pyaniasetools as pya

import batch_optimizers as opt

import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'batch_optimizers'

In [ ]:
mol = hdt.readxyz2('/nh/nest/u/jsmith/Research/confs_test/capsaicin.xyz')
N = 50

wkdir = '/home/jsmith48/scratch/ANI-2x_retrain/model1/'
cns = wkdir + 'rHCNOSFCl-4.6R_16-3.1A_a4-8.params'
sae = wkdir + 'sae_linfit.dat'
nnf = wkdir + 'train'
Nn = 5

#wkdir = '/nh/nest/u/jsmith/scratch/Gits/ANI-Networks/networks/ANI-1x/ANI-9.0.4_netarch8/'
#cns = wkdir + 'rHCNO-4.6A_16-3.1A_a4-8.params'
#sae = wkdir + 'sae_6-31gd.dat'
#nnf = wkdir + 'train'
#Nn = 1

In [ ]:
Xl = np.stack([mol[0][0] for i in range(N)])
S = list(mol[1])
Xl.shape

In [ ]:
X = Xl+np.random.uniform(-0.15,0.15,size=Xl.shape)
X = np.asarray(X, dtype=np.float32)
hdt.writexyzfile('/nh/nest/u/jsmith/Research/confs_test/verify1.xyz',X,S)

In [ ]:
# Set NC
aens = ensemblemolecule(cns, sae, nnf, Nn, 0)

In [ ]:
from ase.parallel import world
from ase import parallel
print(parallel.size)

In [ ]:
start_time = time.time()
for i,x in enumerate(X[0:1]):
    mol = Atoms(S,x)
    mol.set_calculator(ANIENS(aens))
    
    dyn = LBFGS(mol)
    dyn.run(fmax=0.00018)
    print('Molecule',i,'complete')
print('[ANI Total time:', time.time() - start_time, 'seconds]')

In [ ]:
# molopt = opt.moleculeOptimizer(cns, sae, nnf, Nn, 0)

In [ ]:
# start_time = time.time()
# X_it = molopt.optimizeGradientDescent(X,S,alpha=0.0004,convergence=1.0, printer=False)
# X_it = molopt.optimizeConjugateGradient(X_it,S,alpha=0.0001,convergence=0.005, maxsteps=5000, printstep=100, printer=False)
# print('[Complete Optimization Time:',time.time()-start_time,'s]')
# hdt.writexyzfile('/nh/nest/u/jsmith/Research/confs_test/verify_mol1.xyz',np.array(X_it),S)

In [ ]:
cnfopt = opt.conformerOptimizer(cns, sae, nnf, Nn, 0)

In [ ]:
start_time = time.time()
X_it = cnfopt.optimizeGradientDescent(X,S,alpha=0.0004,convergence=10.0, maxsteps=10000, printstep=10)
X_it = cnfopt.optimizeConjugateGradient(X_it,S,alpha=0.0003,convergence=0.005, maxsteps=10000, printstep=10)
print('[Complete Optimization Time:',time.time()-start_time,'s]')
hdt.writexyzfile('/nh/nest/u/jsmith/Research/confs_test/verify_mol2.xyz',np.array(X_it),S)